# Introduction

#market-risk, #var, #variance-covariance, #sbm, #frtb, #cva-frtb, #simm

Run this notebook to create an analytical application for the SBM charge. The input data will be stored in-memory and Atoti will perform the computation "on-the-fly" based on user query. You can filter, drill down and explore your data and the SBM metrics.

## About SBM

You will find below how the SBM logic can be described in python and injected into Atoti. We'll narrow down the use case to Equity Delta charge for simplicity. The other aggregation chains can be added in a similar manner.

Sensitivity-based mathod is one of the parametric market risk methodologies. It can be used to compute a VaR-like metric from sensitivities using pre-calibrated risk weights and correlations, through a sequence of nested variance covariance formulae. Due to its multiple benefits (see [2]), the method is widely used both internally by orgazations to manage market risk, as well as by Regulators for capital requirements purposes (FRTB) as well as margining (SIMM). 

The notebook assumes that the reader is familiar with the sensitivity-based method and terminology.

## What to expect

In this notebook, we'll read the required sensitivities and calculation parameters and create a chain of measures aggregating data:

sensitivities by trade -> weighted sensitivities by risk factor -> charges by bucket -> equity delta margin


## Input data

Input data is sourced in a CRIF-like format - see [3].

## References 

- [1]: Consolidated Basel Framework Chapter MAR21: https://www.bis.org/basel_framework/chapter/MAR/21.htm?inforce=20220101. 
- [2]: ISDA SIMM(TM): From Principles to Model Specification: https://www.isda.org/a/vAiDE/simm-from-principles-to-model-specification-4-mar-2016-v4-public.pdf
- [3]: ISDA Risk Data Standard: https://www.isda.org/a/owEDE/risk-data-standards-v1-36-public.pdf

# Getting started

## Imports

In [1]:
from IPython.display import display, Markdown, Latex, Image
import numpy as np
import pandas as pd
import os

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.max_colwidth", 300)
pd.options.display.float_format = "{:,.4f}".format

## Atoti

In [2]:
import atoti as tt
from atoti.config import create_config

session = tt.create_session(
    config="./configuration.yaml", port="53972", sampling_mode=tt.sampling.FULL
)

Welcome to Atoti 0.3.1!

By using this community edition, you agree with the license available at https://www.atoti.io/eula.
Browse the official documentation at https://docs.atoti.io.
Join the community at https://www.atoti.io/register.

You can hide this message by setting the ATOTI_HIDE_EULA_MESSAGE environment variable to True.


## Url

In [3]:
session.url

'http://localhost:53972'

# Input data files

These are the csv-files that we need to illustrate the Equity Delta aggregation:

- "smaller_data.csv" - sensitivities in a CRIF-like format
- "eq_delta_gamma.csv" - cross-bucket correlations, set for each pair of buckets
- "eq_delta_rho.csv" - risk factor correlations, set per bucket, i.e. all pairs of risk factors in a given bucket have the same correlation
- "eq_delta_rw.csv" - risk weights, set per bucket and per risk factor type (stored in Label2 crif field)

The example data files are stored in a cloud and I'm dowloading them to the working directory.

In [4]:
import sys

sys.path.append("../../utils")


from notebook_utils import download_source

download_source("https://data.atoti.io/notebooks/sbm/smaller_data.csv")
download_source("https://data.atoti.io/notebooks/sbm/bigger_data.csv")
download_source("https://data.atoti.io/notebooks/sbm/parameters/eq_delta_gamma.csv")
download_source("https://data.atoti.io/notebooks/sbm/parameters/eq_delta_rho.csv")
download_source("https://data.atoti.io/notebooks/sbm/parameters/eq_delta_rw.csv")

# Sensitivities datastore

In [5]:
# in this example, the data is initially read into a pandas dataframe,
# which is subsequently used to create a datastore.
crif = pd.read_csv("smaller_data.csv")
crif = crif.append(pd.read_csv("bigger_data.csv"), ignore_index=True)
crif.head(5)

,TradeID,RiskType,Qualifier,Label2,AmountUSD,Bucket,PortfolioID
0,0,Risk_Equity,Wilmar International,REPO,"-10,332.0900",1,Smaller_Portfolio
1,1,Risk_Equity,Wilmar International,SPOT,"-3,641,606.4500",1,Smaller_Portfolio
2,2,Risk_Equity,China Minmetals,REPO,"-2,337.9000",3,Smaller_Portfolio
3,3,Risk_Equity,China Minmetals,SPOT,"-10,549.5400",3,Smaller_Portfolio
4,4,Risk_Equity,China Life Insurance,REPO,"-7,874.9400",4,Smaller_Portfolio


In [6]:
risks_store = session.read_pandas(
    crif,
    keys=["TradeID", "PortfolioID", "RiskType", "Qualifier", "Label2"],
    store_name="Risks",
    types={"Bucket": tt.types.STRING},
)

cube = session.create_cube(risks_store)
lvl = cube.levels
m = cube.measures
h = cube.hierarchies

In [8]:
# creating a comparator to sort buckets as numbers:
# NOT WORKING AT THE MOMENT
# lvl["Bucket"].comparator = tt.comparator.first_members([str(i) for i in range(1,13)])

At this point, a cube has been created, and we can start browsing the sensitivities:

In [8]:
cube.visualize()

Install the Atoti JupyterLab extension to see this widget.

Run this command to create a new data vizualisation:

In [10]:
cube.visualize()

Install the Atoti JupyterLab extension to see this widget.

# Risk weights datastore

In [11]:
# the risk weights data store is created from a csv:
eq_delta_risk_weights_store = session.read_csv(
    "eq_delta_rw.csv",
    keys=["Bucket", "Label2"],
    types={"Bucket": tt.types.STRING},
    store_name="RiskWeights",
)

# risks store is joined with the risk weights store
risks_store.join(eq_delta_risk_weights_store)

In [12]:
eq_delta_risk_weights_store.head(5)

,,RW
Bucket,Label2,
1,SPOT,0.5500
2,SPOT,0.6000
3,SPOT,0.4500
4,SPOT,0.5500
5,SPOT,0.3000


In [15]:
# The risk weights are now available in the cube as a measure.
# The measure is defined for Bucket and Label2 - these hierarchies need to be present into the view.

In [14]:
cube.visualize()

Install the Atoti JupyterLab extension to see this widget.

In [16]:
# Let's put it into a folder and format as percentages:
m["RW.VALUE"].folder = "Parameters"
m["RW.VALUE"].formatter = "DOUBLE[#%]"

In [18]:
cube.visualize()

Install the Atoti JupyterLab extension to see this widget.

# Weighted sensitivities

In this section, we'll create a measure to compute weighted sensitivities defined in
[MAR21.4](https://www.bis.org/basel_framework/chapter/MAR/21.htm?inforce=20220101#paragraph_MAR_21_20220101_21_4):

$$WS_k=RW_k \cdot s_k$$

As the risk weights are defined for each Bucket and Label2, this is the level where sensitivities need to be multiplied by the risk weight.


In [19]:
# The input sensitivities are multplied by the risk weight for each Bucket and Label2,
# and then summed up to obtain weighted sensitivities:
m["WS"] = tt.agg.sum(
    m["AmountUSD.SUM"] * m["RW.VALUE"], scope=tt.scope.origin("Bucket", "Label2")
)

In [21]:
cube.visualize()

Install the Atoti JupyterLab extension to see this widget.

# Bucket-level aggregation

The weighted sensitivities by risk factor are rolled up into charges by bucket ("bucket-level charges"), using a variance-covariance-type of formula that can be found in the [MAR21.4](https://www.bis.org/basel_framework/chapter/MAR/21.htm?inforce=20220101#paragraph_MAR_21_20220101_21_4):

$$K_{b} =\sqrt{max \left( 0, \sum _{k\in b} WS_{k}^{2} +\sum _{k\in b}\sum
  _{l\in b, l\neq k}\rho_{kl}\cdot WS_k \cdot WS_l\right)}$$
  
In this section we'll create measures visualizing bucket-level charges. 

We will provide two methods to compute them:

1. Method 1: materializing both risk factors in a pair and looping over all the pais of the risk factors - $\color{red}{\text{this is not efficient, }O(N^2)}$.
2. Method 2: more computationally efficient: leveraging the fact, that some of the risk factor pairs are correlated with the same correlation parameter, we provide a more computationally efficient calculation. 

## Risk factor correlations

The parameter $\rho_{kl}$ denotes correlation between two risk factors $k$ and $l$ in a pair of risk factors. The rules defining the equity delta correlations are set in [MAR21.78](https://www.bis.org/basel_framework/chapter/MAR/21.htm?inforce=20220101#paragraph_MAR_21_20220101_21_78).

The rules can be summarised for each pair of risk factors as follows:

- Case 1: same name, different type: a single value -> 0.999
- Case 2: different name, same type: a single value depending on bucket, for example, 0.15
- Case 3: different name, different type: value depending on risk factor multiplier by 0.999, for example, 0.15 x 0.999

Stylized example:


| risk factors | name1-spot | name1-repo | name2-spot | name2-repo |
|------------|-------------|-------------|-------------|-------------|
| name1-spot | 1 |  |  |  |
| name1-repo | same_name_diff_type | 1 |  |  |
| name2-spot | rho_by_name | rho_by_name x type_multiplier | 1 |  |
| name2-repo | rho_by_name x type_multiplier | rho_by_name x type_multiplier | same_name_diff_type | 1 |

In [22]:
# Equity delta risk factor is defined as a combination of fields - "Qualifier" and "Label2", i.e. equity name and risk factor type.
# Creating variables:
same_risk_factor = 1.0
same_name_diff_type = 0.999
diff_type_multiplier = 0.999

In [23]:
# Creating a datastore holding correlations defined per bucket ([MAR21.78](2)):
eq_delta_rho = session.read_csv(
    "eq_delta_rho.csv",
    keys=["Bucket"],
    types={"Bucket": tt.types.STRING},
    store_name="RiskFactorCorrelations",
)
risks_store.join(eq_delta_rho)
eq_delta_rho.head(5)

,names_correlation
Bucket,
1,0.1500
2,0.1500
3,0.1500
4,0.1500
5,0.2500


In [24]:
# Let's put it into a folder and format as percentages:
m["names_correlation.VALUE"].folder = "Parameters"
m["names_correlation.VALUE"].formatter = "DOUBLE[0.00]"

In [26]:
cube.visualize()

Install the Atoti JupyterLab extension to see this widget.

## Method 1: materializing risk factor pairs

In [28]:
# Important:
# We join by PortfolioID - hence
# the combinations of risk factors are restricted to those that are defined below PortfolioID.

In [29]:
# Now, for each risk factor - "Qualifier" + "Label2" - there's a list of all risk factors in portfolio and bucket.
# The hierarchies representing risk factors: "Other Qualifier", "Other Label2".
other_risk_factor_store = session.read_pandas(
    crif[["PortfolioID", "Bucket", "Qualifier", "Label2"]].rename(
        columns={"Qualifier": "Other Qualifier", "Label2": "Other Label2"}
    ),
    keys=["PortfolioID", "Bucket", "Other Qualifier", "Other Label2"],
    types={"Bucket": tt.types.STRING},
    store_name="OtherRiskFactor",
)
risks_store.join(
    other_risk_factor_store, mapping={"PortfolioID": "PortfolioID", "Bucket": "Bucket"}
)

In [30]:
# After creating the "other_risk_factor_store", for each portfolio, bucket and risk factor ->
# the cube contains the list of all risk factors that belong to the same bucket.

In [31]:
# Setting up a measure returning correlation for any pair of risk factors.
# Risk factors are represented by the hierarchies ("Qualifier", "Label2") and ("Other Qualifier", "Other Label2")
m["rho_kl"] = tt.where(
    lvl["Qualifier"] == lvl["Other Qualifier"],
    tt.where(
        lvl["Label2"] == lvl["Other Label2"], same_risk_factor, same_name_diff_type
    ),
    tt.where(
        lvl["Label2"] == lvl["Other Label2"],
        m["names_correlation.VALUE"],
        m["names_correlation.VALUE"] * diff_type_multiplier,
    ),
)

The following measures will represent $WS_k$ and $WS_l$ in the Bucket-level aggregation formula:

In [32]:
m["WSk"] = m["WS"]
m["WSl"] = tt.at(
    m["WS"],
    {lvl["Qualifier"]: lvl["Other Qualifier"], lvl["Label2"]: lvl["Other Label2"]},
)

# Please note, that for the bucket 11, bucket-level charge is defined as sum of absoluted
# weighted sensitivities by risk factor [MAR21.79](https://www.bis.org/basel_framework/chapter/MAR/21.htm?inforce=20220101#paragraph_MAR_21_20220101_21_79)

m["Kb"] = tt.agg.stop(
    tt.where(
        lvl["Bucket"] == "11",
        tt.agg.sum(tt.abs(m["WSk"]), tt.scope.origin("Qualifier", "Label2")),
        tt.sqrt(
            tt.max(
                0,
                tt.agg.sum(
                    m["WSk"] * m["WSl"] * m["rho_kl"],
                    scope=tt.scope.origin(
                        "Qualifier", "Other Qualifier", "Label2", "Other Label2"
                    ),
                ),
            )
        ),
    ),
    at=[lvl["Bucket"]],
)

In [61]:
cube.query(m['WS'], m['WSk'], m['WSl'], levels = [lvl['Bucket'], lvl['Qualifier'], lvl['Label2'], lvl['Other Qualifier'], lvl['Other Label2']], condition = lvl['PortfolioID']=="Smaller_Portfolio")

WS  \
Bucket Qualifier            Label2 Other Qualifier      Other Label2                   
1      Wilmar International REPO   Wilmar International REPO                -56.8265   
                                                        SPOT                -56.8265   
                            SPOT   Wilmar International REPO         -2,002,883.5475   
                                                        SPOT         -2,002,883.5475   
10     AB Volvo             REPO   AB Volvo             REPO                  3.5524   
...                                                                              ...   
9      Severstal            SPOT   Lukoil               SPOT             54,174.2250   
                                   SAIC Motor           REPO             54,174.2250   
                                                        SPOT             54,174.2250   
                                   Severstal            REPO             54,174.2250   
                                                        SPOT             54,174.2250   

                                                                                 WSk  \
Bucket Qualifier            Label2 Other Qualifier      Other Label2                   
1      Wilmar International REPO   Wilmar International REPO                -56.8265   
                                                        SPOT                -56.8265   
                            SPOT   Wilmar International REPO         -2,002,883.5475   
                                                        SPOT         -2,002,883.5475   
10     AB Volvo             REPO   AB Volvo             REPO                  3.5524   
...                                                                              ...   
9      Severstal            SPOT   Lukoil               SPOT             54,174.2250   
                                   SAIC Motor           REPO             54,174.2250   
                                                        SPOT             54,174.2250   
                                   Severstal            REPO             54,174.2250   
                                                        SPOT             54,174.2250   

                                                                                 WSl  
Bucket Qualifier            Label2 Other Qualifier      Other Label2                  
1      Wilmar International REPO   Wilmar International REPO                -56.8265  
                                                        SPOT         -2,002,883.5475  
                            SPOT   Wilmar International REPO                -56.8265  
                                                        SPOT         -2,002,883.5475  
10     AB Volvo             REPO   AB Volvo             REPO                  3.5524  
...                                                                              ...  
9      Severstal            SPOT   Lukoil               SPOT             78,087.5760  
                                   SAIC Motor           REPO                106.0724  
                                                        SPOT            547,661.1000  
                                   Severstal            REPO                 -7.5913  
                                                        SPOT             54,174.2250  

[1068 rows x 3 columns]

In [33]:
cube.query(
    m["AmountUSD.SUM"],
    m["WS"],
    m["Kb"],
    levels=[lvl["Bucket"]],
    condition=lvl["PortfolioID"] == "Smaller_Portfolio",
)

,AmountUSD.SUM,WS,Kb
Bucket,,,
1,"-3,651,938.5400","-2,002,940.3740","2,002,940.3172"
10,"-5,652,387.6950","-2,810,353.8262","1,819,315.9010"
11,"9,197,909.9830","6,418,348.7082","6,418,348.7082"
12,"-1,913,082.0100","-286,992.4351","286,992.4354"
3,"-12,887.4400","-4,757.8136","4,757.8031"
4,"-1,030,083.3000","-557,071.4497","758,465.4868"
5,"1,443,720.0700","429,349.4284","456,861.0617"
7,"239,581.2030","91,493.4106","932,682.8928"
8,"-10,423,086.7700","-5,177,444.7259","3,937,788.9195"


## Method 2: optimized Kb calculation

Since many of the risk factor pairs share the same correlation value, it is possible to optimize the variance-covariance aggregation. The efficiency of this calculation is critical when the data cardinality along risk factor is high.

We decompose the formula into the three components:
1. contribution of pairs with both risk factors being spot - same correlation $\rho_{names}$ set per bucket,
2. contribution of pairs with both risk factors being repo - same correlation $\rho_{names}$ set per bucket,
3. contribution of pairs where one risk factor is spot, another one is repo - same correlation $\rho_{names} \cdot 0.999$. We'd need to account that for some of the pairs where one risk factor is spot, one is repo some the equity names will match and need to be correlated at 0.999


Let's start with the pairs where both risk factors are either spot or repo - cases 1. and 2. above. Since for any $k$ and $l$ the correlation $\rho_{kl}$ will be equal to correlation defined per bucket $\rho_{names}$, their contribution can be rewritten:

$$ \sum_k WS_k^2 +  \sum_k \sum_{l \neq k} \rho_{kl} WS_k WS_l = \\ \sum_k WS_k^2 + \rho_{names} \cdot \left(\left( \sum_k WS_k \right)^2 - \sum_k WS_k^2  \right) = \\ (1-\rho_{names}) \cdot \sum_k WS_k^2 + \rho_{names} \cdot \left( \sum_k WS_k \right)^2  \label{reduced_formula} \tag{reduced_formula}$$

In [34]:
# This measure will display sum of WS_k squared by risk factor (Qualifier + Label2):
m["sum squares"] = tt.agg.square_sum(
    m["WS"], scope=tt.scope.origin("Qualifier", "Label2")
)

In [35]:
reduced_formula = (1 - m["names_correlation.VALUE"]) * m["sum squares"] + m[
    "names_correlation.VALUE"
] * tt.pow(m["WS"], 2.0)

# Total contribution of:
# - pairs having only spot risk factors,
# - pairs having only repo risk factors,
# is the sum of the reduced formula by Label2 members.

m["spot&repo pairs contribution"] = tt.agg.sum(
    reduced_formula, scope=tt.scope.origin("Label2")
)

The contribution of the risk factors where one risk factor belongs to "SPOT" and the other belongs to "REPO" - case 3 above - can be rewritten:
$$ \sum_k WS_k^2 +  \sum_k \sum_{l \neq k} \rho_{kl} WS_k WS_l = \\ \vec{WS_{repo}^T} \cdot J_{n_{repo}, n_{spot}} \cdot \vec{WS_{spot}} \cdot \rho_{names} \cdot 0.999 + 0.999 \cdot (1 - \rho_{names}) \sum_{n\in names}{WS_n^{repo} \cdot WS_n^{spot}} \label{spot_vs_repo} $$

where:
- J - is a matrix of ones,
- first term in the above formula performs aggregation of all sensitivities, as if they all are correlated at $\rho_{names}$,
- the second term is to correct the first term and to account for the fact that risk factors, where spot and repo risk factors have the same equity name, must be correlated at 0.999.


**First term**

The measure `sum WS_repo |J| WS_spot` will display $\vec{WS_{repo}^T} \cdot J_{n_{repo}, n_{spot}} \cdot \vec{WS_{spot}} $.

In [36]:
# filtering for repo and spot risk factors:
m["WS_spot"] = tt.filter(m["WSk"], lvl["Label2"] == "SPOT")
m["WS_repo"] = tt.filter(m["WSk"], lvl["Label2"] == "REPO")

# Collect WS of the spot risk factors in a vector and show against every qualifier in a bucket:
weights_vector = tt.agg._vector(m["WS_spot"], tt.scope.origin("Bucket", "Qualifier"))
m["spot vector"] = tt.parent_value(
    weights_vector, on_hierarchies=[h["Qualifier"], h["Label2"]]
)

# cross product of weighted sensitivities for the names, having both spot and repo sensitivities:
# Multiply vector WS of the spot risk factors by WS of each repo risk factor sum them up
repo_scalar_x_spot_vector = m["WS_repo"] * m["spot vector"]
m["sum WS_repo |J| WS_spot"] = tt.agg.stop(
    tt.agg.sum(
        tt.array.sum(repo_scalar_x_spot_vector),
        scope=tt.scope.origin("Bucket", "Qualifier"),
    ),
    at=[lvl["Bucket"]],
)

**Second term**

The measure `sum WS repo and spot` will display $\sum_{n\in names}{WS_n^{repo} \cdot WS_n^{spot}}$

In [37]:
######### Below formula will multiply by null - might be a bad style??

In [38]:
m["WS_spot_parent"] = tt.agg.sum(m["WS_spot"], scope=tt.scope.origin("Label2"))
m["WS_repo_parent"] = tt.agg.sum(m["WS_repo"], scope=tt.scope.origin("Label2"))
m["sum WS repo and spot"] = tt.agg.stop(
    tt.agg.sum(
        m["WS_spot_parent"] * m["WS_repo_parent"], scope=tt.scope.origin("Qualifier")
    ),
    at=[lvl["Bucket"]],
)

**Combined result cross spot & repo pairs**

In [39]:
# final contribution of the pairs where one risk factor is spot, one is repo
m["cross repo spot contribution"] = (
    m["names_correlation.VALUE"] * m["sum WS_repo |J| WS_spot"] * 0.999
    + 0.999 * (1 - m["names_correlation.VALUE"]) * m["sum WS repo and spot"]
)

**Total Kb**

In [40]:
m["Kb alternative"] = tt.where(
    lvl["Bucket"] == "11",
    tt.agg.sum(tt.abs(m["WSk"]), tt.scope.origin("Qualifier", "Label2")),
    tt.sqrt(
        tt.max(
            0, m["spot&repo pairs contribution"] + 2 * m["cross repo spot contribution"]
        )
    ),
)

In [41]:
cube.query(
    m["Kb"],
    m["Kb alternative"],
    levels=lvl["Bucket"],
    condition=lvl["PortfolioID"] == "Smaller_Portfolio",
)

,Kb,Kb alternative
Bucket,,
1,"2,002,940.3172","2,002,940.3172"
10,"1,819,315.9010","1,819,315.9010"
11,"6,418,348.7082","6,418,348.7082"
12,"286,992.4354","286,992.4354"
3,"4,757.8031","4,757.8031"
4,"758,465.4868","758,465.4868"
5,"456,861.0617","456,861.0617"
7,"932,682.8928","932,682.8928"
8,"3,937,788.9195","3,937,788.9195"


# Cross-bucket aggregation

## Bucket correlations

In [42]:
eq_delta_buckets_correlations = session.read_csv(
    "eq_delta_gamma.csv",
    keys=["Bucket", "Other Bucket"],
    types={"Bucket": tt.types.STRING, "Other Bucket": tt.types.STRING},
    store_name="eq_delta_corr_outer",
)
risks_store.join(eq_delta_buckets_correlations)

In [65]:
cube.visualize()

Install the Atoti JupyterLab extension to see this widget.

## Aggregating across buckets

In [43]:
# 21.4(5)(a):
m["WSb"] = m["WS"]
m["WSc"] = tt.at(m["WS"], {lvl["Bucket"]: lvl["Other Bucket"]})
m["sum Kb2 + sum sum WSb WSc gamma"] = tt.agg.square_sum(
    m["Kb"], tt.scope.origin("Bucket")
) + tt.agg.sum(
    m["WSb"] * m["WSc"] * m["gamma.VALUE"],
    scope=tt.scope.origin("Bucket", "Other Bucket"),
)

# 21.4(5)(b):
m["Sb"] = tt.max(tt.min(m["WS"], m["Kb"]), -1.0 * m["Kb"])
m["Sc"] = tt.at(m["Sb"], {lvl["Bucket"]: lvl["Other Bucket"]})
m["sum Kb2 + sum sum Sb Sc gamma"] = tt.agg.square_sum(
    m["Kb"], tt.scope.origin(lvl["Bucket"])
) + tt.agg.sum(
    m["Sb"] * m["Sc"] * m["gamma.VALUE"],
    scope=tt.scope.origin("Bucket", "Other Bucket"),
)

m["Delta Margin"] = tt.where(
    m["sum Kb2 + sum sum WSb WSc gamma"] > 0,
    tt.sqrt(m["sum Kb2 + sum sum WSb WSc gamma"]),
    tt.sqrt(m["sum Kb2 + sum sum Sb Sc gamma"]),
)

In [44]:
cube.query(
    m["Delta Margin"], condition=lvl["PortfolioID"] == "Smaller_Portfolio", timeout=90
)

,Delta Margin
0,"8,951,153.7749"


$\color{red}{\text{This calculation goes out of memory}}$

In [ ]:
cube.query(
    m["Delta Margin"], condition=lvl["PortfolioID"] == "Bigger_Portfolio", timeout=120
)

In [54]:
# It goes out of memory because there're many risk factors:

In [66]:
crif[crif.PortfolioID=="Smaller_Portfolio"][['Qualifier', 'Label2']].drop_duplicates().shape[0]

86

In [55]:
crif[crif.PortfolioID=="Bigger_Portfolio"][['Qualifier', 'Label2']].drop_duplicates().shape[0]

1571

In [56]:
# Optimized calculation

# 21.4(5)(a):
m["sum Kb2 + sum sum WSb WSc gamma alternative"] = tt.agg.square_sum(
    m["Kb alternative"], scope=tt.scope.origin("Bucket")
) + tt.agg.sum(
    m["WSb"] * m["WSc"] * m["gamma.VALUE"], tt.scope.origin("Bucket", "Other Bucket")
)

# 21.4(5)(b):
m["Sb alternative"] = tt.max(
    tt.min(m["WSk"], m["Kb alternative"]), -1.0 * m["Kb alternative"]
)
m["Sc alternative"] = tt.at(m["Sb alternative"], {lvl["Bucket"]: lvl["Other Bucket"]})
m["sum Kb2 + sum sum Sb Sc gamma alternative"] = tt.agg.square_sum(
    m["Kb alternative"], scope=tt.scope.origin("Bucket")
) + tt.agg.sum(
    m["Sb alternative"] * m["Sc alternative"] * m["gamma.VALUE"],
    scope=tt.scope.origin("Bucket", "Other Bucket"),
)

m["Delta Margin alternative"] = tt.where(
    m["sum Kb2 + sum sum WSb WSc gamma alternative"] > 0,
    tt.sqrt(m["sum Kb2 + sum sum WSb WSc gamma alternative"]),
    tt.sqrt(m["sum Kb2 + sum sum Sb Sc gamma alternative"]),
)

In [57]:
cube.query(m["Delta Margin alternative"], levels=[lvl["PortfolioID"]])

,Delta Margin alternative
PortfolioID,
Bigger_Portfolio,"9,357,880.5093"
Smaller_Portfolio,"8,951,153.7749"


# Timeit

In [58]:
%%timeit
cube.query(
    m["Delta Margin"], condition=lvl["PortfolioID"] == "Smaller_Portfolio", timeout=90
)

108 ms ± 15.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit
cube.query(
    m["Delta Margin"], condition=lvl["PortfolioID"] == "Bigger_Portfolio", timeout=120
)

In [59]:
%%timeit
cube.query(
    m["Delta Margin alternative"], condition=lvl["PortfolioID"] == "Smaller_Portfolio"
)

122 ms ± 66.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [60]:
%%timeit
cube.query(
    m["Delta Margin alternative"], condition=lvl["PortfolioID"] == "Bigger_Portfolio"
)

85.7 ms ± 8.02 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
